### Método de Newton

In [26]:
import numpy as np
from sympy import Symbol, diff

In [27]:
def create_F(number_of_functions, functions):
    F = np.array([[functions[i]] for i in range(number_of_functions)])
    return F

In [28]:
def create_J(number_of_functions, number_of_constants, functions):
    J = np.empty((number_of_functions, number_of_constants), dtype=object)
    for i in range(number_of_functions):
        for j in range(number_of_constants):
            J[i][j] = diff(functions[i], f"c{j+2}")
    return J

In [29]:
def update_c_dict(X, extra_dict={}):
    c_dict = {f"c{i+2}":X[i, 0] for i in range(len(X))}
    c_dict.update(extra_dict)
    return c_dict

In [30]:
def sub_F(F, c_dict):
    F = F.copy()
    for i in range(F.shape[0]):
        for j in range(F.shape[1]):
            F[i][j] = F[i][j].subs(c_dict)
    return F.astype(float)

In [31]:
def sub_J(J, c_dict):
    J = J.copy()
    for i in range(J.shape[0]):
        for j in range(J.shape[1]):
            J[i][j] = J[i][j].subs(c_dict)
    return J.astype(float)

In [118]:
c2 = Symbol('c2')
c3 = Symbol('c3')
c4 = Symbol('c4')
θ1 = Symbol('θ1')
θ2 = Symbol('θ2')

f1 = 2*c3**2+c2**2+6*c4**2-1
f2 = 8*c3**3+6*c3*c2**2+36*c3*c2*c4+108*c3*c4**2-θ1
f3 = 60*c3**4+60*c3**2*c2**2+576*c3**2*c2*c4+2232*c3**2*c4**2+252*c4**2*c2**2+1296*c4**3*c2+3348*c4**4+24*c2**3*c4+3*c2-θ2

fns = [f1, f2, f3]

number_of_functions = 3
number_of_constants = 3

In [119]:
# c2 = Symbol("c2")
# c3 = Symbol("c3")
# θ1 = Symbol('θ1')
# θ2 = Symbol('θ2')

In [120]:
# f1 = c2 + 2*c3 - 2
# f1

In [121]:
# f2 = c2**2 + 4*c3**2 - 4
# f2

In [122]:
# fns = [f1, f2]

# number_of_functions = 2
# number_of_constants = 2


In [161]:
max_iter = 100
max_tol = 1e-6
theta_dict = {"θ1": 0, "θ2": 3}

In [131]:
F = create_F(number_of_functions, fns)
F.shape

(3, 1)

In [132]:
J = create_J(number_of_functions, number_of_constants, fns)
J.shape

(3, 3)

In [133]:
J

array([[2*c2, 4*c3, 12*c4],
       [12*c2*c3 + 36*c3*c4, 6*c2**2 + 36*c2*c4 + 24*c3**2 + 108*c4**2,
        36*c2*c3 + 216*c3*c4],
       [72*c2**2*c4 + 120*c2*c3**2 + 504*c2*c4**2 + 576*c3**2*c4 + 1296*c4**3 + 3,
        120*c2**2*c3 + 1152*c2*c3*c4 + 240*c3**3 + 4464*c3*c4**2,
        24*c2**3 + 504*c2**2*c4 + 576*c2*c3**2 + 3888*c2*c4**2 + 4464*c3**2*c4 + 13392*c4**3]],
      dtype=object)

In [195]:
def calculate_nl_broyden(max_iter, max_tol, theta_dict, X0, F, J):
    X_arr = np.empty((max_iter, ), dtype="object")
    B_arr = np.empty((max_iter, ), dtype="object")
    X_arr[0] = X0

    B_arr[0] = sub_J(J, {"c2": X_arr[0][0][0], "c3": X_arr[0][1][0], "c4": X_arr[0][2][0]} | theta_dict)

    for k in range(1, max_iter+1):
        if k == max_iter:
            raise Exception("Não convergiu")
        J_ = B_arr[k-1]
        delta_X = - np.matmul(np.linalg.inv(J_), sub_F(F, {"c2": X_arr[k-1][0][0], "c3": X_arr[k-1][1][0], "c4": X_arr[k-1][2][0]} | theta_dict))
        X_arr[k] = X_arr[k-1] + delta_X
        Y_k = sub_F(F, {"c2": X_arr[k][0][0], "c3": X_arr[k][1][0], "c4": X_arr[k][2][0]} | theta_dict) - sub_F(F, {"c2": X_arr[k-1][0][0], "c3": X_arr[k-1][1][0], "c4": X_arr[k-1][2][0]} | theta_dict)
        tolk = np.linalg.norm(delta_X)/np.linalg.norm(X_arr[k])
        if tolk < max_tol:
            return X_arr[k]
        else:
            B_arr[k] = B_arr[k-1] + ((Y_k - (B_arr[k-1] @ delta_X)) @ (delta_X.T))/((delta_X.T) @ delta_X)

In [196]:
X0 = np.array([[0.4], [0], [-0.5]])

In [197]:
calculate_nl_broyden(max_iter, max_tol, theta_dict, X0, F, J)

array([[ 0.89062614],
       [ 0.        ],
       [-0.18564729]])

In [129]:
tolk

7.615324947913931e-11

In [191]:
def run_broyden(max_iter, max_tol, X0, theta_dict):
    c2 = Symbol('c2')
    c3 = Symbol('c3')
    c4 = Symbol('c4')
    θ1 = Symbol('θ1')
    θ2 = Symbol('θ2')
    
    f1 = 2*c3**2+c2**2+6*c4**2-1
    f2 = 8*c3**3+6*c3*c2**2+36*c3*c2*c4+108*c3*c4**2-θ1
    f3 = 60*c3**4+60*c3**2*c2**2+576*c3**2*c2*c4+2232*c3**2*c4**2+252*c4**2*c2**2+1296*c4**3*c2+3348*c4**4+24*c2**3*c4+3*c2-θ2
    
    fns = [f1, f2, f3]
    
    number_of_functions = 3
    number_of_constants = 3
    
    F = create_F(number_of_functions, fns)
    J = create_J(number_of_functions, number_of_constants, fns)

    X = calculate_nl_broyden(max_iter, max_tol, theta_dict, X0, F, J)
    return np.round(X.ravel(), 3).tolist()

In [192]:
max_iter = 100
max_tol = 1e-4
X0 = np.array([1, 0, -0.2]).reshape(-1, 1)
theta_dict = {"θ1": 0, "θ2": 3}

In [193]:
c2, c3, c4 = run_broyden(max_iter, max_tol, X0, theta_dict)

In [194]:
c2, c3, c4

(0.891, 0.0, -0.185)

In [ ]:
    c2 = Symbol('c2')
    c3 = Symbol('c3')
    c4 = Symbol('c4')
    θ1 = Symbol('θ1')
    θ2 = Symbol('θ2')
    
    f1 = 2*c3**2+c2**2+6*c4**2-1
    f2 = 8*c3**3+6*c3*c2**2+36*c3*c2*c4+108*c3*c4**2-θ1
    f3 = 60*c3**4+60*c3**2*c2**2+576*c3**2*c2*c4+2232*c3**2*c4**2+252*c4**2*c2**2+1296*c4**3*c2+3348*c4**4+24*c2**3*c4+3*c2-θ2

In [ ]:
f2.subs({"c2": 0.784, "c3": 0.25, "c4": -0.208} | theta_dict)